In [ ]:
from pathlib import Path
import re
import json
import itertools
from collections import Counter
import requests

import pypandoc
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
%matplotlib inline

In [ ]:
def load_txt_competence_profile(file_path: Path) -> str:
    with file_path.open('r') as file_handle:
        return file_handle.readlines()

# Preparation
## Download the competence profiles and peer information
- Download and extract the folder of competence profiles from gDrive, rename it to "competence_profiles" and put it at a place of your choice.
- Adjust `COMPETENCE_PROFILES_DICT_PATH` to point to this folder.
- Go to prisma, open F12, hit `peers`, open the `graphql` response and paste the json into a file in this folder and give it the name `prisma_peers_graphql.json` as the constant `PRISMA_PEERS_JSON_RESPONSE_FILE_PATH` below.

## Configuration

In [ ]:
COMPETENCE_PROFILES_DICT_PATH = Path('/Users/matt/competence_profiles')
PRISMA_PEERS_JSON_RESPONSE_FILE_PATH = Path('prisma_peers_graphql.json')
HARCODED_FILTER_WORDS = ['architektur', 'technologien', 'f', '.']

# Data Extraction and Filtering

In [ ]:
def get_all_short_names_of_files(path_to_folder: Path) -> list[str]:
    """Given all the competence profile files at the path_to_folder location, extract all ipt short names from it."""
    file_paths = path_to_folder.glob('Competence Profile_???.docx')
    
    def extract_shortname_from_file_name(file_name: str) -> str:
        return file_name.split('_')[1].split('.')[0]
    
    short_names = [extract_shortname_from_file_name(path.name) for path in file_paths]
    print(f'Found {len(short_names)} competence profiles under {path_to_folder}')
    return list(short_names)

def convert_competence_profile_to_plain_text(competence_profile_file_path: Path) -> str:
    """This will convert a .docx competence profile to simple text and removes weird characters originating from tables."""
    
    txt = pypandoc.convert_file(str(competence_profile_file_path), 'org')
    txt = txt.split('\n', 1)[1]  # remove first line (comes from profile image)
    
    # The following filters are necessary since there are a lot of weird symbols due to the tables in the word document in the text.
    txt = txt.replace('|', ' ')
    txt = txt.replace('-', ' ')
    txt = txt.replace('=', ' ')
    txt = txt.replace('+', ' ')
    txt = txt.replace('*', ' ')
    txt = txt.replace('/', ' ')
    txt = txt.replace('# begin_quote', ' ')
    txt = txt.replace('# end_quote', ' ')
    txt = txt.replace('\n', ' ')
    txt = txt.strip()
    return txt
    
def extract_all_words(competence_profile_plain_txt: str) -> list[str]:
    """"Given plain text, extract all the words contained in this unstructured text."""
    
    regex_pattern = r"\(\d\s+('?\w+)"
    return re.findall("[a-zA-ZöäüÖÜÄ\-\.'/]+", competence_profile_plain_txt)

def extract_core_competence_words(competence_profile_words: list[str]) -> list[str]:
    """This will filter the input words such that only words after 'Kernkompetenzen' and before 'ipt Projekte' will remain."""
    
    arrived_at_core_competences = False
    core_competence_words = []
    for idx, word in enumerate(competence_profile_words):
        if word == 'Kernkompetenzen':
            arrived_at_core_competences = True
            continue
        if arrived_at_core_competences:
            if word == 'ipt' and competence_profile_words[idx + 1] == 'Projekte':
                break
            else:
                core_competence_words.append(word)
    return core_competence_words

def to_lower(words: list[str]) -> list[str]:
    return [word.lower() for word in words]

def filter_out_stop_words(words: list[str]) -> list[str]:
    sp = spacy.load('de_core_news_sm')
    stop_words = sp.Defaults.stop_words
    return [word for word in words if word.lower() not in stop_words]

def build_competence_dict(base_path: Path) -> dict[str, list[str]]:
    """
    Given a base_path pointing to a folder were all the .docx competence profiles reside, build a a competence_dict.
    
    The competence_dict's structure is as follows
    {'ABC': ['masta_skill', 'epic_skill'], ...}
    """
    short_names = get_all_short_names_of_files(BASE_PATH)
    competence_dict = {short_name: [] for short_name in short_names}

    for idx, short_name in tqdm(enumerate(short_names), total=len(short_names)):
        file_path = BASE_PATH / f'Competence Profile_{short_name}.docx'
        txt = convert_competence_profile_to_plain_text(file_path)
        words = extract_all_words(txt)
        core_competence_words = extract_core_competence_words(words)
        core_competence_words = filter_out_stop_words(core_competence_words)
        core_competence_words = to_lower(core_competence_words)
        core_competence_words = [word for word in core_competence_words if word not in HARCODED_FILTER_WORDS]
        competence_dict[short_name] = core_competence_words
    
    return competence_dict

def load_peers_from_prisma_response(prisma_response_json_file: Path) -> list[dict]:
    with open(prisma_response_json_file) as infile:
        json_response = json.loads(infile.read())
        user_info_list = []
        for user in json_response['data']['listUsers']:
            user_info = {
                'firstname': user.get('firstname', ''),
                'lastname': user.get('lastname', ''),
                'abbreviation': user.get('abbreviation', ''),
                'picture': user.get('picture', '')
            }
            if user_info['firstname'] == None:
                continue
            user_info_list.append(user_info)
        return user_info_list
    
def enrich_prisma_data_with_seniority_and_start_date(user_info_list: list[dict]) -> list[dict]:
    base_url = 'https://ipt.ch/de/team/mitarbeiter'
    for iptler in tqdm(user_info_list):
        firstname = iptler["firstname"].strip().lower().replace('ä', 'a').replace('ö', 'o').replace('ü', 'u')
        lastname = iptler["lastname"].strip().lower().replace('ä', 'a').replace('ö', 'o').replace('ü', 'u')
        full_url = f'{base_url}/{firstname}-{lastname}'
        response = requests.get(full_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # parse seniority
            seniority = soup.find("p", class_="team-detail--role")
            iptler['seniority'] = seniority.text.strip().replace('-', ' ')
            
            # parse starting year
            table_heading = soup.find("th", class_="team-detail--table-heading", text="Start bei ipt")

            if table_heading:
                # Navigate to the corresponding <td> tag
                table_data = table_heading.find_next_sibling("td", class_="team-detail--table-field")
                if table_data:
                    # Extract the text content of the table data
                    start_year = table_data.text.strip()
                    iptler['start_year'] = start_year
    return user_info_list

def find_iptlers_with_skills(iptlers: list[dict], competence_dicts: dict[list], skills: list, min_seniority: str = None, print_out: bool = False) -> list[dict]:
    seniority_order = {
        0: ['Junior Software Engineer'],
        1: ['Consultant'],
        2: ['Senior Consultant'],
        3: ['IT Architect', 'Lead Consultant'],
        4: ['Principal Architect', 'Principal Consultant'],
        5: ['Principal Architect, Director', 'Principal Consultant, Director'],
        6: ['Associate Partner'],
        7: ['Partner']
    }
    all_seniority_levels = sum(seniority_order.values(), [])

    
    def get_seniority_level(job_title: str) -> int:
        for level, titles in seniority_order.items():
            if job_title in titles:
                return level
        raise ValueError(f'Seniority level {job_title} invalid.')
    
    
    if min_seniority is not None and min_seniority not in all_seniority_levels:
        raise ValueError(f'min_seniority must be one of {all_seniority_levels}')
    
    matches = []
    for iptler in iptlers:
        if min_seniority is not None:
            if 'seniority' in iptler.keys():  # only have it if on website
                if iptler['seniority'] not in all_seniority_levels:
                    continue # e.g. Marketing
                if get_seniority_level(min_seniority) > get_seniority_level(iptler['seniority']):
                    continue
        
        matching_skills = []
        if iptler['abbreviation'].upper() not in competence_dict.keys():
            continue
        for skill in skills:
            if skill in competence_dicts[iptler['abbreviation'].upper()]:
                matching_skills.append(skill)
        if len(matching_skills) > 0:
            iptler['skill_matches'] = matching_skills
            matches.append(iptler)
        matches.sort(key=lambda match: len(match['skill_matches']), reverse=True)
    if print_out:
        seniority_statement = '' if min_seniority is None else f' and seniority above (including) {min_seniority}'
        print(f'Found {len(matches)} IPTlers with matching skills{seniority_statement}:')
        print(10 * '-')
        
        for match in matches:
            for key, value in match.items():
                if key == 'picture': continue
                if key == 'skill_matches':
                    value = 'Skill matches: ' + f' '.join(value)
                if key == 'abbreviation':
                    value = value.upper()
                if key == 'start_year':
                    value = f'Start IPT: {value}'
                print(f'{value}')
            print(5 * '-')
    return matches

# Visualization

In [ ]:
def generate_word_cloud_from_competence_dict(competence_dict: dict) -> None:
    """Given the competence_dict this will generate and store a word cloud with word sizes weighted on occurrence."""
    
      # These sort of clutter everythin
    all_words = list(itertools.chain(*competence_dict.values())) 
    
    all_words = [word for word in all_words if word not in HARCODED_FILTER_WORDS]
    wordcloud = WordCloud(width = 1600, height = 1000,
                background_color ='white',
                min_font_size = 10).generate(' '.join(all_words))
 
    # plot the WordCloud image                      
    plt.figure(figsize = (15, 15), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.savefig('ipt_competence_word_cloud.png', bbox_inches='tight')
    plt.show()


def create_bar_plot_most_common_skills(competence_dict: dict) -> None:
    sns.set_style("whitegrid", {'axes.grid' : False})

    # Initialize the matplotlib figure
    f, ax = plt.subplots(figsize=(6, 12))

    # Load the example car crash dataset
    counter = Counter(list(itertools.chain(*competence_dict.values())))
    skill_df = pd.DataFrame(counter.most_common(), columns=['skill', 'occurrences'])
    skill_df = skill_df.loc[skill_df['occurrences'] > 75]
    skill_df['normed_occurrence'] = skill_df['occurrences'] / max(skill_df['occurrences'])
    
    pal = sns.color_palette("Greens_d", len(skill_df))
    sns.barplot(x="normed_occurrence", 
                y="skill", 
                data=skill_df,
                orient = "h")
    ax.set(xlabel='Normierte Häufigkeit', ylabel='Skill')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    
    plt.savefig('ipt_skill_occurrence_distribution.png', bbox_inches='tight')

# Executable Code Section

The following section needs (MacOS) `brew install pandoc` and from within your virtual environment `python -m spacy download de_core_news_sm`.

In [ ]:
competence_dict = build_competence_dict(COMPETENCE_PROFILES_DICT_PATH)

In [ ]:
generate_word_cloud_from_competence_dict(competence_dict)

In [ ]:
create_bar_plot_most_common_skills(competence_dict)

In [ ]:
iptlers = load_peers_from_prisma_response('prisma_peers_graphql.json')
iptlers = enrich_prisma_data_with_seniority_and_start_date(iptlers)

In [ ]:
skills = ['kubernetes', 'openshift']

matches = find_iptlers_with_skills(iptlers, competence_dict, skills, min_seniority='Consultant', print_out=True)